In [1]:
import pandas as pd
import requests
import math 
from decimal import Decimal 

In [6]:
symbole = "SOLUSDT"
url = "https://api.binance.com/api/v3/depth"
intervale = Decimal('0.1')
paramètres =  { 
    "symbol": symbole, 
    "limit":1000, 
} 
data = requests.get(url, paramètres).json() 
data

{'lastUpdateId': 10907242503,
 'bids': [['136.15000000', '36.33000000'],
  ['136.14000000', '66.76000000'],
  ['136.13000000', '21.41000000'],
  ['136.12000000', '148.87000000'],
  ['136.11000000', '119.49000000'],
  ['136.10000000', '210.82000000'],
  ['136.09000000', '292.76000000'],
  ['136.08000000', '141.02000000'],
  ['136.07000000', '372.08000000'],
  ['136.06000000', '162.37000000'],
  ['136.05000000', '336.28000000'],
  ['136.04000000', '196.71000000'],
  ['136.03000000', '508.32000000'],
  ['136.02000000', '617.75000000'],
  ['136.01000000', '226.58000000'],
  ['136.00000000', '277.97000000'],
  ['135.99000000', '220.80000000'],
  ['135.98000000', '102.05000000'],
  ['135.97000000', '716.61000000'],
  ['135.96000000', '326.62000000'],
  ['135.95000000', '87.71000000'],
  ['135.94000000', '247.63000000'],
  ['135.93000000', '564.11000000'],
  ['135.92000000', '208.28000000'],
  ['135.91000000', '887.23000000'],
  ['135.90000000', '213.49000000'],
  ['135.89000000', '137.300000

In [8]:
#Créer les Dataframes pour le côté d'achat (bid)
bid_levels = pd.DataFrame(data["bids"], columns = ['price', 'quantity'], dtype = float)
bid_levels["side"] = "Achat"
bid_levels


,price,quantity,side
0,136.15,36.33,Achat
1,136.14,66.76,Achat
2,136.13,21.41,Achat
3,136.12,148.87,Achat
4,136.11,119.49,Achat
...,...,...,...
995,126.18,10.91,Achat
996,126.17,3.96,Achat
997,126.16,14.64,Achat
998,126.15,6.64,Achat


In [9]:
#Créer les DataFrames pour le côté de vente (ask) 
ask_levels = pd.DataFrame(data["asks"], columns = ['price', 'quantity'], dtype = float)
ask_levels["side"] = "vente" 
ask_levels 

,price,quantity,side
0,136.16,470.95,vente
1,136.17,283.99,vente
2,136.18,249.79,vente
3,136.19,315.21,vente
4,136.20,575.93,vente
...,...,...,...
995,146.12,0.40,vente
996,146.13,0.83,vente
997,146.14,1.10,vente
998,146.15,7.89,vente


In [10]:
#Afficher les 5 meilleures offres de vente (les prix les plus bas) 
meilleures_offres_vente = ask_levels.nsmallest(5, 'price')
print("Les 5 meilleures offres (vente) :")
print(meilleures_offres_vente)

Les 5 meilleures offres (vente) :
    price  quantity   side
0  136.16    470.95  vente
1  136.17    283.99  vente
2  136.18    249.79  vente
3  136.19    315.21  vente
4  136.20    575.93  vente


In [11]:
# Afficher les 5 meilleures demandes d'achat (les prix les plus hauts)
meilleures_demandes_achat = bid_levels.nlargest(5, 'price')
print("Les 5 meilleures demandes (achat) :")
print(meilleures_demandes_achat)

Les 5 meilleures demandes (achat) :
    price  quantity   side
0  136.15     36.33  Achat
1  136.14     66.76  Achat
2  136.13     21.41  Achat
3  136.12    148.87  Achat
4  136.11    119.49  Achat


In [12]:
#obtenir le nombre d'ordres proposés à un prix donné
nombre_ordres_par_prix = bid_levels.groupby('price').size().reset_index(name='nombre_ordres')
print("Nombre d'ordres proposés à chaque prix (Achat) :")
print(nombre_ordres_par_prix)

Nombre d'ordres proposés à chaque prix (Achat) :
      price  nombre_ordres
0    126.14              1
1    126.15              1
2    126.16              1
3    126.17              1
4    126.18              1
..      ...            ...
995  136.11              1
996  136.12              1
997  136.13              1
998  136.14              1
999  136.15              1

[1000 rows x 2 columns]


In [14]:
carnet_ordre = pd.concat([meilleures_demandes_achat,meilleures_offres_vente])
carnet_ordre

,price,quantity,side
0,136.15,36.33,Achat
1,136.14,66.76,Achat
2,136.13,21.41,Achat
3,136.12,148.87,Achat
4,136.11,119.49,Achat
0,136.16,470.95,vente
1,136.17,283.99,vente
2,136.18,249.79,vente
3,136.19,315.21,vente
4,136.20,575.93,vente


In [28]:
def is_taker_function(carnet_ordre, nouvel_ordre):
    if nouvel_ordre['side'].iloc[0] == 'Achat':
        # Pour un achat, vérifie si un ordre de vente correspondant existe au même prix ou moins
        return not carnet_ordre[(carnet_ordre['side'] == 'Vente') & (carnet_ordre['price'] <= nouvel_ordre['price'].iloc[0])].empty
    else:
        # Pour une vente, vérifie si un ordre d'achat correspondant existe au même prix ou plus
        return not carnet_ordre[(carnet_ordre['side'] == 'Achat') & (carnet_ordre['price'] >= nouvel_ordre['price'].iloc[0])].empty

def executer_ordre_comme_taker(carnet_ordre, ordre_taker):
    côté_opposé = 'Achat' if ordre_taker['side'].iloc[0] == 'Vente' else 'Vente'
    ordres_opposés = carnet_ordre[(carnet_ordre['side'] == côté_opposé)]
    ordres_opposés = ordres_opposés.sort_values(by='price', ascending=(côté_opposé == 'Vente'))
    
    quantité_à_exécuter = ordre_taker['quantity'].iloc[0]

    for index, ordre_opposé in ordres_opposés.iterrows():
        if quantité_à_exécuter <= 0:
            break
        quantité_executée = min(quantité_à_exécuter, ordre_opposé['quantity'])
        carnet_ordre.at[index, 'quantity'] -= quantité_executée
        quantité_à_exécuter -= quantité_executée

        if carnet_ordre.at[index, 'quantity'] <= 0:
            carnet_ordre.drop(index, inplace=True)

    carnet_ordre.reset_index(drop=True, inplace=True)
    return carnet_ordre

def ajouter_nouvel_ordre(carnet_ordre, nouvel_ordre_dict):
    # Créez un DataFrame pour le nouvel ordre
    nouvel_ordre = pd.DataFrame([nouvel_ordre_dict])

    # Vérifiez si l'ordre est un Taker
    is_taker = is_taker_function(carnet_ordre, nouvel_ordre)

    # Si c'est un Taker, exécutez-le contre le carnet
    if is_taker:
        carnet_ordre = executer_ordre_comme_taker(carnet_ordre, nouvel_ordre)
    else:
        # Sinon, ajoutez-le au carnet en tant que Maker
        carnet_ordre = pd.concat([carnet_ordre, nouvel_ordre], ignore_index=True)

    return carnet_ordre

# Exemple d'utilisation de la fonction ajouter_nouvel_ordre
nouvel_ordre_dict = {
    'price': 136.15,  # Le prix de l'ordre
    'quantity': 36.33,  # La quantité de l'ordre
    'side': 'Vente'  # Le côté de l'ordre, 'Achat' ou 'Vente'
}

# Ajoutez le nouvel ordre au carnet
carnet_ordre = ajouter_nouvel_ordre(carnet_ordre, nouvel_ordre_dict)

# Affichez le carnet d'ordres mis à jour pour vérifier que le nouvel ordre a été ajouté correctement
print(carnet_ordre)


     price  quantity   side
0   136.14     66.76  Achat
1   136.13     21.41  Achat
2   136.12    148.87  Achat
3   136.11    119.49  Achat
4   136.16    470.95  vente
5   136.17    283.99  vente
6   136.18    249.79  vente
7   136.19    315.21  vente
8   136.20    575.93  vente
9   100.50      5.00  Achat
10  100.50      5.00  Achat
11  136.15     36.33  Achat
12  101.50     10.00  Achat
13  136.15     36.33  Achat
